In [1]:
# import the necessary packages
import pandas as pd
import re

In [2]:
# make the merging simpler by:
def clean_song(row):
    song = row.str.lower() # making all names lowercased
    song = str(song.str.split(' -')[0][0]) # taking out anything after - (like "- Radio Edition")
    song = re.sub(r" \(feat. .+(?= .tayl)", "", song) # take out anything with "feature"
    song = re.sub(r" \(feat. .+(?= \()", "", song) # take out extra parentheses
    song = re.sub(r" \(feat. .+", "", song) # take out remaining lines after feature
    song = song.replace("’", "'") # replace all ’ with '
    return song

In [3]:
# read in Spotify data
swift_1 = pd.read_table('all_songs.csv')
swift_1 = swift_1.rename(columns={"name": "song"}) # rename columns to match
swift_1.song = swift_1.apply(clean_song, axis=1)

In [7]:
# read in Billboard Hot 100 data
swift_2 = pd.read_table("../billboardHot100/long_df_swift.txt")
swift_2 = swift_2[swift_2['artist'].str.contains("Taylor Swift")]
swift_2['date'] = swift_2['date'].astype('datetime64[D]')
swift_2 = pd.pivot(data=swift_2, index="song", columns=['date'], values='rank')
swift_2 = swift_2.reset_index()
swift_2.song = swift_2.apply(clean_song, axis=1)
swift_2 = swift_2.set_index(swift_2.song)
swift_2 = swift_2.drop(columns=['song'])

In [8]:
# join the tables on "song"
a = swift_1.join(swift_2, on='song', how='outer').reset_index()

,index,song,album_name,release_date,popularity,duration,danceability,2006-09-23 00:00:00,2006-09-30 00:00:00,2006-10-07 00:00:00,...,2022-02-12 00:00:00,2022-02-19 00:00:00,2022-02-26 00:00:00,2022-03-05 00:00:00,2022-03-12 00:00:00,2022-05-21 00:00:00,2022-07-09 00:00:00,2022-11-05 00:00:00,2022-11-12 00:00:00,2022-11-19 00:00:00
0,0.0,lavender haze,Midnights (3am Edition),2022-10-22,88.0,202395.0,0.735,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,23.0
1,1.0,maroon,Midnights (3am Edition),2022-10-22,88.0,218270.0,0.658,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,12.0,36.0
2,2.0,anti-hero,Midnights (3am Edition),2022-10-22,89.0,200690.0,0.638,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,3.0,snow on the beach,Midnights (3am Edition),2022-10-22,87.0,256124.0,0.659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,16.0,40.0
4,4.0,"you're on your own, kid",Midnights (3am Edition),2022-10-22,88.0,194206.0,0.694,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,20.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,NaN,highway don't care,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,i don't wanna live forever (fifty shades darker),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,NaN,renegade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,NaN,safe & sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# remove duplicate songs (keep only the most popular)
index = -1
to_remove = []
count = a['song'].value_counts()

for i in range(len(a)):
    song = a.iloc[i].song
    if count[song] > 1:
        if i + 1 < len(a):
            current_row = a.iloc[i]
            next_row = a.iloc[i+1]
            if current_row.song == next_row.song:
                if current_row.popularity < next_row.popularity:
                    to_remove.append(i)
                else:
                    to_remove.append(i+1)

b = a.drop(to_remove)

In [12]:
# melt and set index to date (one row for every song + date combination)
c = b.drop(columns=['index']).reset_index().drop(columns=['index'])
d = pd.melt(c, id_vars=c.columns[:6], var_name = 'date', value_name = 'rank')
f = d.dropna()
f.index = f.set_index('date').index.astype('datetime64[ns]')

In [13]:
# save as a csv
f.to_csv('date_indexed_songs.csv', sep='\t', index=False)